In [30]:
# Import Libraries:
from datetime import datetime, timedelta # Datetime Feature
import os
import pandas as pd # Import Pandas
import panel as pn # Import Panel
import plotly.express as px # Import Plotly
pn.extension("plotly")# Import Plotly Extension
from pprint import pprint
import requests # To fetch data from our url API

# Import and initialize coinmetrics API (in this case `cm` for the Coinmetrics Community API)
import coinmetrics # Coinmetrics API
cm = coinmetrics.Community()

import alpaca_trade_api as tradeapi

In [2]:
# API INPUTS
asset = "btc"
begin_timestamp = (datetime.now() - timedelta(days=90)).date()
end_timestamp = (datetime.now() - timedelta(days=0)).date()
# end_timestamp = datetime.now().date()


# API OUTPUT (Coinmetrics Community URL API)
asset_data = requests.get(f"https://community-api.coinmetrics.io/v4/timeseries/market-candles?start_time={begin_timestamp}&end_time={end_timestamp}&paging_from=start&markets=coinbase-{asset}-usd-spot&pretty=true").json()
# pprint(asset_data)



In [32]:
# Load .env enviroment variables
from dotenv import load_dotenv
load_dotenv('../../classkeys.env')

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# YOUR CODE HERE!
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2"
)

In [49]:
# Set the ticker
ticker = "SPY"
          
# Set timeframe to '1D'
timeframe = "1D"

# Set start and end datetimes to same as coinmetrics data 
start_date = pd.Timestamp(begin_timestamp.isoformat(), tz="America/New_York").isoformat()
end_date = pd.Timestamp(end_timestamp.isoformat(), tz="America/New_York").isoformat()

# Get BTC duration of data for SPY
ticker_data = alpaca.get_barset(
    ticker,
    timeframe,
    start=start_date,
    end=end_date,
    limit=1000,
).df


ticker_data.head()

SPY                               
                            open   high    low  close    volume
time                                                           
2021-08-19 00:00:00-04:00 436.27 441.14 436.12 439.84  84838725
2021-08-20 00:00:00-04:00 440.23 443.71 439.71 443.33  63120064
2021-08-23 00:00:00-04:00 445.16 448.23 443.44 447.26  49349485
2021-08-24 00:00:00-04:00 447.97 448.54 447.42 447.95  33180117
2021-08-25 00:00:00-04:00 448.17 449.46 447.77 448.88  35262175

In [58]:
# Initiate clean dictionary
data_dictionary = {
    "date" : [],
    "open" : [],
    "close" : [],
    "high" : [],
    "low" : [],
    "volume" : [],
    "vwap" : [], 
}

# Use For Loop to interate through data and populate dictionary
for entry in asset_data['data']:
    data_dictionary['date'].append(entry['time'])
    data_dictionary['open'].append(float(entry['price_open']))
    data_dictionary['close'].append(float(entry['price_close']))
    data_dictionary['high'].append(float(entry['price_high']))
    data_dictionary['low'].append(float(entry['price_low']))
    data_dictionary['volume'].append(float(entry['volume']))
    data_dictionary['vwap'].append(float(entry['vwap']))

#Convert dictionary into Pandas dataframe and format to two decimals
asset_data_df = pd.DataFrame.from_dict(data_dictionary)
pd.options.display.float_format = "{:,.2f}".format
# INSERT HERE >>> FIND WAY TO CLEAN DATE DATA

# View dataframe
asset_data_df.head()


date      open     close      high       low  \
0  2021-08-19T00:00:00.000000000Z 44,709.10 46,770.14 47,088.08 43,955.00   
1  2021-08-20T00:00:00.000000000Z 46,769.02 49,342.40 49,400.00 46,644.95   
2  2021-08-21T00:00:00.000000000Z 49,349.99 48,867.02 49,821.92 48,285.28   
3  2021-08-22T00:00:00.000000000Z 48,870.21 49,284.63 49,526.13 48,102.89   
4  2021-08-23T00:00:00.000000000Z 49,284.38 49,506.50 50,505.00 49,012.84   

     volume      vwap  
0 11,938.12 45,486.35  
1 13,184.67 48,000.67  
2  9,574.84 49,058.09  
3  6,243.40 48,847.07  
4 10,824.27 49,881.69

## 

# Rolling Average

In [12]:
date_asset_df = asset_data_df.set_index("date")
date_asset_df

open     close      high       low  \
date                                                                     
2021-08-19T00:00:00.000000000Z 44,709.10 46,770.14 47,088.08 43,955.00   
2021-08-20T00:00:00.000000000Z 46,769.02 49,342.40 49,400.00 46,644.95   
2021-08-21T00:00:00.000000000Z 49,349.99 48,867.02 49,821.92 48,285.28   
2021-08-22T00:00:00.000000000Z 48,870.21 49,284.63 49,526.13 48,102.89   
2021-08-23T00:00:00.000000000Z 49,284.38 49,506.50 50,505.00 49,012.84   
...                                  ...       ...       ...       ...   
2021-11-13T00:00:00.000000000Z 64,144.42 64,400.01 64,990.00 63,394.17   
2021-11-14T00:00:00.000000000Z 64,400.01 65,505.02 65,525.00 63,596.90   
2021-11-15T00:00:00.000000000Z 65,505.03 63,624.59 66,339.90 63,371.00   
2021-11-16T00:00:00.000000000Z 63,621.05 60,107.98 63,624.60 58,638.00   
2021-11-17T00:00:00.000000000Z 60,088.65 60,351.51 60,824.38 58,380.00   

                                  volume      vwap  
date                                                
2021-08-19T00:00:00.000000000Z 11,938.12 45,486.35  
2021-08-20T00:00:00.000000000Z 13,184.67 48,000.67  
2021-08-21T00:00:00.000000000Z  9,574.84 49,058.09  
2021-08-22T00:00:00.000000000Z  6,243.40 48,847.07  
2021-08-23T00:00:00.000000000Z 10,824.27 49,881.69  
...                                  ...       ...  
2021-11-13T00:00:00.000000000Z  5,968.10 64,159.70  
2021-11-14T00:00:00.000000000Z  6,081.57 64,515.20  
2021-11-15T00:00:00.000000000Z 10,426.56 64,753.40  
2021-11-16T00:00:00.000000000Z 24,910.15 60,821.42  
2021-11-17T00:00:00.000000000Z 20,907.45 59,824.24  

[91 rows x 6 columns]

In [56]:
sp_btc_df = pd.concat([date_asset_df, ticker_data], axis="rows", join="outer")
sp_btc_df

TypeError: concat() got an unexpected keyword argument 'parse_dates'

### Calculate Daily Returns

In [18]:
daily_returns = date_asset_df["close"].pct_change().dropna().to_frame()
daily_returns.head()

close
date                                 
2021-08-20T00:00:00.000000000Z   0.05
2021-08-21T00:00:00.000000000Z  -0.01
2021-08-22T00:00:00.000000000Z   0.01
2021-08-23T00:00:00.000000000Z   0.00
2021-08-24T00:00:00.000000000Z  -0.04

### Evaluate Riskiness of Stocks

In [25]:
import numpy as np

In [26]:
# Use the `std` function and multiply by the square root of the number of trading days in a year to get annualized volatility
volatility = daily_returns.std() * np.sqrt(365)
volatility.sort_values(inplace=True)
volatility

close   0.67
dtype: float64

In [13]:
asset_data_rolling_30day = asset_data_df['close'].rolling(window=30).mean()
print(asset_data_rolling)


0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
        ...   
86   60,727.40
87   61,147.15
88   61,555.62
89   61,803.12
90   62,096.54
Name: close, Length: 91, dtype: float64


# RSI - Alex

# BOLLINGER BANDS - Alex

# MAC D - Saydee

# V Wap - Saydee

# Rolling Averages - Jeff

# SKDJ - Stochastic Oscillator - Anjan

# NVT - Network Value to Transaction

# The Puell Multiple - Brian / Jeff

# Sharpe Ratios

In [9]:
# Risk free rate doc saves in resources
# SAMPLE: risk_free_rate = 0.0000022
# combined_sharpe_ratios = ((combined_returns.mean()-(risk_free_rate)) * 252) / (combined_returns.std() * np.sqrt(252))
